In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import LogNormalAFTFitter, WeibullAFTFitter, LogLogisticAFTFitter
from lifelines import CoxPHFitter


In [2]:
# get data from s3
url = "https://s3.amazonaws.com/survival2024/hurricane.csv"
df = pd.read_csv(url)

In [3]:
df['flood']=0
df.loc[df['reason'] == 1, 'flood'] = 1
df_flood = df[['backup', 'age', 'bridgecrane', 'servo', 'gear', 'trashrack', 'slope', 'elevation', 'hour', 'flood']]
df_flood.head()

,backup,age,bridgecrane,servo,gear,trashrack,slope,elevation,hour,flood
0,0,6.0,0,0,0,1,3,2,48,0
1,0,6.0,1,0,0,1,7,3,48,0
2,0,6.1,1,0,0,1,3,3,48,0
3,1,6.1,1,0,0,0,4,3,48,0
4,0,6.2,1,1,0,1,3,4,48,0


## AFT Model

### Weibull
has the highest log-likelihood ratio test, thus is the best fit for our data

In [4]:
aft_Weibull = WeibullAFTFitter()
aft_Weibull.fit(df_flood, duration_col='hour', event_col='flood',ancillary=False )
aft_Weibull.print_summary()

<lifelines.WeibullAFTFitter: fitted with 770 total observations, 655 right-censored observations>
             duration col = 'hour'
                event col = 'flood'
   number of observations = 770
number of events observed = 115
           log-likelihood = -724.73
         time fit was run = 2024-11-11 14:55:02 UTC

---
                     coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param   covariate                                                                                                    
lambda_ age          0.05      1.05      0.07           -0.09            0.18                0.92                1.20
        backup       0.25      1.28      0.12            0.00            0.49                1.00                1.63
        bridgecrane -0.19      0.83      0.20           -0.57            0.20                0.56                1.22
        elevation    0.05      1.05      0.08           -0.10            0.21                0.90                1.23
        gear         0.39      1.47      0.25           -0.09            0.87                0.91                2.39
        servo        0.27      1.31      0.14           -0.01            0.54                0.99                1.72
        slope       -0.06      0.94      0.02           -0.10           -0.03                0.91                0.97
        trashrack   -0.24      0.78      0.12           -0.49            0.00                0.61                1.00
        Intercept    4.57     96.50      0.57            3.46            5.68               31.73              293.45
rho_    Intercept    0.45      1.56      0.09            0.28            0.62                1.32                1.85

                     cmp to     z      p  -log2(p)
param   covariate                                 
lambda_ age            0.00  0.71   0.48      1.07
        backup         0.00  1.97   0.05      4.37
        bridgecrane    0.00 -0.94   0.34      1.54
        elevation      0.00  0.68   0.50      1.00
        gear           0.00  1.58   0.11      3.12
        servo          0.00  1.90   0.06      4.14
        slope          0.00 -3.50 <0.005     11.08
        trashrack      0.00 -1.95   0.05      4.30
        Intercept      0.00  8.05 <0.005     50.14
rho_    Intercept      0.00  5.21 <0.005     22.37
---
Concordance = 0.67
AIC = 1469.46
log-likelihood ratio test = 40.09 on 8 df
-log2(p) of ll-ratio test = 18.31

### Log Logistic

In [5]:
aft_LogLogistic = LogLogisticAFTFitter()
aft_LogLogistic.fit(df_flood, duration_col='hour', event_col='flood')
aft_LogLogistic.print_summary()

<lifelines.LogLogisticAFTFitter: fitted with 770 total observations, 655 right-censored observations>
             duration col = 'hour'
                event col = 'flood'
   number of observations = 770
number of events observed = 115
           log-likelihood = -724.71
         time fit was run = 2024-11-11 14:55:04 UTC

---
                    coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param  covariate                                                                                                    
alpha_ age          0.04      1.04      0.07           -0.10            0.18                0.90                1.19
       backup       0.25      1.29      0.13            0.00            0.51                1.00                1.66
       bridgecrane -0.20      0.82      0.20           -0.60            0.19                0.55                1.21
       elevation    0.05      1.05      0.08           -0.11            0.20                0.89                1.23
       gear         0.41      1.50      0.24           -0.07            0.88                0.93                2.42
       servo        0.32      1.37      0.14            0.04            0.60                1.04                1.81
       slope       -0.06      0.94      0.02           -0.10           -0.03                0.90                0.97
       trashrack   -0.26      0.77      0.13           -0.51           -0.00                0.60                1.00
       Intercept    4.54     93.91      0.59            3.39            5.70               29.55              298.46
beta_  Intercept    0.51      1.66      0.08            0.34            0.67                1.40                1.96

                    cmp to     z      p  -log2(p)
param  covariate                                 
alpha_ age            0.00  0.53   0.60      0.75
       backup         0.00  1.98   0.05      4.38
       bridgecrane    0.00 -1.00   0.32      1.66
       elevation      0.00  0.56   0.58      0.80
       gear           0.00  1.68   0.09      3.44
       servo          0.00  2.21   0.03      5.20
       slope          0.00 -3.35 <0.005     10.26
       trashrack      0.00 -1.98   0.05      4.38
       Intercept      0.00  7.70 <0.005     46.05
beta_  Intercept      0.00  5.97 <0.005     28.61
---
Concordance = 0.67
AIC = 1469.42
log-likelihood ratio test = 40.58 on 8 df
-log2(p) of ll-ratio test = 18.61

### Log Normal

In [6]:
aft_LogNormalAFTFitter = LogNormalAFTFitter()
aft_LogNormalAFTFitter.fit(df_flood, duration_col='hour', event_col='flood')
aft_LogNormalAFTFitter.print_summary()

<lifelines.LogNormalAFTFitter: fitted with 770 total observations, 655 right-censored observations>
             duration col = 'hour'
                event col = 'flood'
   number of observations = 770
number of events observed = 115
           log-likelihood = -728.09
         time fit was run = 2024-11-11 14:55:04 UTC

---
                    coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param  covariate                                                                                                    
mu_    age         -0.00      1.00      0.08           -0.16            0.16                0.85                1.17
       backup       0.31      1.36      0.15            0.02            0.60                1.02                1.82
       bridgecrane -0.27      0.76      0.23           -0.73            0.18                0.48                1.20
       elevation    0.01      1.01      0.09           -0.17            0.19                0.84                1.21
       gear         0.52      1.68      0.26           -0.00            1.03                1.00                2.81
       servo        0.42      1.53      0.16            0.10            0.74                1.11                2.10
       slope       -0.06      0.94      0.02           -0.11           -0.02                0.90                0.98
       trashrack   -0.28      0.75      0.15           -0.57            0.01                0.56                1.01
       Intercept    5.20    181.40      0.68            3.88            6.53               48.20              682.65
sigma_ Intercept    0.26      1.30      0.08            0.11            0.41                1.12                1.51

                    cmp to     z      p  -log2(p)
param  covariate                                 
mu_    age            0.00 -0.00   1.00      0.00
       backup         0.00  2.12   0.03      4.87
       bridgecrane    0.00 -1.18   0.24      2.06
       elevation      0.00  0.11   0.91      0.13
       gear           0.00  1.96   0.05      4.32
       servo          0.00  2.60   0.01      6.73
       slope          0.00 -2.71   0.01      7.22
       trashrack      0.00 -1.92   0.05      4.19
       Intercept      0.00  7.69 <0.005     45.96
sigma_ Intercept      0.00  3.48 <0.005     10.95
---
Concordance = 0.67
AIC = 1476.18
log-likelihood ratio test = 37.58 on 8 df
-log2(p) of ll-ratio test = 16.76

### Feature Selection w/ Weibull. 
Backwards Selection at the alpha = 0.03 level

In [7]:
aft_Weibull.fit(df_flood.loc[:, df_flood.columns != 'age'], duration_col='hour', event_col='flood',ancillary=False )
aft_Weibull.fit(df_flood.loc[:, ~df_flood.columns.isin(['age', 'elevation'])], duration_col='hour', event_col='flood',ancillary=False )
aft_Weibull.fit(df_flood.loc[:, ~df_flood.columns.isin(['age', 'elevation', 'bridgecrane'])], duration_col='hour', event_col='flood',ancillary=False )
aft_Weibull.fit(df_flood.loc[:, ~df_flood.columns.isin(['age', 'elevation', 'bridgecrane', 'gear'])], duration_col='hour', event_col='flood',ancillary=False )
aft_Weibull.fit(df_flood.loc[:, ~df_flood.columns.isin(['age', 'elevation', 'bridgecrane', 'gear', 'trashrack'])], duration_col='hour', event_col='flood',ancillary=False )

aft_Weibull.print_summary()

<lifelines.WeibullAFTFitter: fitted with 770 total observations, 655 right-censored observations>
             duration col = 'hour'
                event col = 'flood'
   number of observations = 770
number of events observed = 115
           log-likelihood = -729.17
         time fit was run = 2024-11-11 14:55:07 UTC

---
                   coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param   covariate                                                                                                  
lambda_ backup     0.27      1.31      0.12            0.03            0.51                1.03                1.67
        servo      0.39      1.47      0.13            0.13            0.64                1.14                1.90
        slope     -0.06      0.94      0.02           -0.09           -0.03                0.91                0.97
        Intercept  4.77    118.05      0.15            4.47            5.07               87.57              159.15
rho_    Intercept  0.44      1.55      0.09            0.27            0.61                1.31                1.83

                   cmp to     z      p  -log2(p)
param   covariate                               
lambda_ backup       0.00  2.19   0.03      5.14
        servo        0.00  2.95 <0.005      8.32
        slope        0.00 -3.47 <0.005     10.93
        Intercept    0.00 31.31 <0.005    712.34
rho_    Intercept    0.00  5.10 <0.005     21.46
---
Concordance = 0.67
AIC = 1468.34
log-likelihood ratio test = 31.21 on 3 df
-log2(p) of ll-ratio test = 20.31

## Cox regression
The varriable `servo` fails the assumptions


In [8]:
cph = CoxPHFitter()
#we drop age here or else assumptions fail in the next step
cph.fit(df_flood.loc[:, df_flood.columns != 'age'], duration_col='hour', event_col='flood')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 770 total observations, 655 right-censored observations>
             duration col = 'hour'
                event col = 'flood'
      baseline estimation = breslow
   number of observations = 770
number of events observed = 115
   partial log-likelihood = -714.57
         time fit was run = 2024-11-11 14:55:08 UTC

---
             coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                    
backup      -0.40      0.67      0.19           -0.77           -0.03                0.46                0.97
bridgecrane  0.28      1.32      0.31           -0.33            0.88                0.72                2.41
servo       -0.46      0.63      0.21           -0.86           -0.06                0.42                0.95
gear        -0.62      0.54      0.38           -1.37            0.12                0.25                1.13
trashrack    0.36      1.44      0.19           -0.01            0.74                0.99                2.10
slope        0.09      1.10      0.03            0.04            0.15                1.04                1.16
elevation   -0.08      0.92      0.12           -0.32            0.16                0.73                1.17

             cmp to     z      p  -log2(p)
covariate                                 
backup         0.00 -2.10   0.04      4.81
bridgecrane    0.00  0.90   0.37      1.44
servo          0.00 -2.23   0.03      5.28
gear           0.00 -1.64   0.10      3.31
trashrack      0.00  1.90   0.06      4.13
slope          0.00  3.46 <0.005     10.86
elevation      0.00 -0.67   0.50      0.99
---
Concordance = 0.67
Partial AIC = 1443.15
log-likelihood ratio test = 38.67 on 7 df
-log2(p) of ll-ratio test = 18.75

In [9]:
cph.check_assumptions(df_flood.loc[:, df_flood.columns != 'age'], p_value_threshold=0.05)#, show_plots=True)

The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 770 total observations, 655 right-censored observations>
         test_name = proportional_hazard_test

---
                  test_statistic    p  -log2(p)
backup      km              0.00 0.96      0.06
            rank            0.01 0.91      0.13
bridgecrane km              1.26 0.26      1.93
            rank            1.24 0.26      1.92
elevation   km              5.94 0.01      6.08
            rank            5.94 0.01      6.07
gear        km              0.06 0.80      0.31
            rank            0.11 0.74      0.43
servo       km              4.57 0.03      4.94
            rank            4.36 0.04      4.76
slope       km              0.12 0.73      0.46
            rank            0.21 0.65      0.63
trashrack   km              0.01 0.90      0.15
            rank            0.01 0.91      0.14



1. Variable 'servo' failed the non-proportional test: p-value is 0.0325.

   Advice: with so few unique values (only 2), you can include `strata=['servo', ...]` in the call
in `.fit`. See documentation in link [E] below.

2. Variable 'elevation' failed the non-proportional test: p-value is 0.0148.

   Advice: with so few unique values (only 5), you can include `strata=['elevation', ...]` in the
call in `.fit`. See documentation in link [E] below.

---
[A]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html
[B]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Bin-variable-and-stratify-on-it
[C]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Introduce-time-varying-covariates
[D]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Modify-the-functional-form
[E]  https://lifelines.re

[]

In [10]:
#backwards selection at alpha 0.03
cph.fit(df_flood.loc[:, ~df_flood.columns.isin(['age', 'elevation', 'bridgecrane', 'gear', 'trashrack'])], duration_col='hour', event_col='flood')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 770 total observations, 655 right-censored observations>
             duration col = 'hour'
                event col = 'flood'
      baseline estimation = breslow
   number of observations = 770
number of events observed = 115
   partial log-likelihood = -718.65
         time fit was run = 2024-11-11 14:55:08 UTC

---
           coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                  
backup    -0.42      0.66      0.19           -0.79           -0.05                0.45                0.95
servo     -0.59      0.55      0.20           -0.98           -0.21                0.38                0.81
slope      0.09      1.10      0.03            0.04            0.15                1.04                1.16

           cmp to     z      p  -log2(p)
covariate                               
backup       0.00 -2.21   0.03      5.20
servo        0.00 -3.02 <0.005      8.63
slope        0.00  3.44 <0.005     10.76
---
Concordance = 0.64
Partial AIC = 1443.30
log-likelihood ratio test = 30.51 on 3 df
-log2(p) of ll-ratio test = 19.83

In [11]:
cph.check_assumptions(df_flood.loc[:, ~df_flood.columns.isin(['age', 'elevation', 'bridgecrane', 'gear', 'trashrack'])], p_value_threshold=0.05)#, show_plots=True)

The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 770 total observations, 655 right-censored observations>
         test_name = proportional_hazard_test

---
             test_statistic    p  -log2(p)
backup km              0.07 0.79      0.33
       rank            0.10 0.75      0.42
servo  km              4.17 0.04      4.60
       rank            4.03 0.04      4.49
slope  km              0.39 0.53      0.90
       rank            0.54 0.46      1.12



1. Variable 'servo' failed the non-proportional test: p-value is 0.0412.

   Advice: with so few unique values (only 2), you can include `strata=['servo', ...]` in the call
in `.fit`. See documentation in link [E] below.

---
[A]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html
[B]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Bin-variable-and-stratify-on-it
[C]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Introduce-time-varying-covariates
[D]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Modify-the-functional-form
[E]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Stratification



[]

## Analyze the pumps that failed due to flood failure and evaluate which pumps to recommend getting an upgrade on the factors found significant by the model.

- You can only perform one upgrade per pump.
- You have a fixed budget of $2.5 million and must use this money accordingly to keep 
  pumps functioning as long as possible.

Our models both found backup, servo, and slope to be the biggest causes of flood failure.
- Slope in non-upgradeable
- Servo costs $150K to upgrade, but has a much lower p-value
- Backup costs $100K to upgrade


In [ ]:
print(f"{1-df_flood[(df_flood['flood']==1)]['servo'].mean():.0%}", "of flood failures did not have a servo.")
print(f"{1-df_flood[(df_flood['flood']==1)]['backup'].mean():.0%}", "of flood failures did not have a backup.")

55% of flood failures did not have a servo.
58% of flood failures did not have a backup.
